# DistilBERT PEFT Training

## Prepare imports

In [1]:
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.metrics import classification_report
import evaluate
import numpy as np
import torch
import pandas as pd

## 1. Loading the pre-processed dataset

In [2]:
# Load the dataset from your local path
df = pd.read_csv('./df1_cleaned_processed.csv')  
df["generated"] = df["generated"].astype(int)

## 2. Train-Test Split

In [3]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Step 1: Split using sklearn (this supports stratify even on float/int)
df_train, df_test = train_test_split(
    df[['text', 'generated']].rename(columns={'generated': 'label'}),
    test_size=0.2,
    stratify=df['generated'],
    random_state=42
)

# Step 2: Convert to Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "test": Dataset.from_pandas(df_test.reset_index(drop=True))
})

dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 85142
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21286
    })
})

## 3. Tokenize the text

In [4]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

tokenized = dataset.map(tokenize, batched=True)
tokenized = tokenized.remove_columns(["text"])
tokenized.set_format("torch")


Map:   0%|          | 0/85142 [00:00<?, ? examples/s]

Map:   0%|          | 0/21286 [00:00<?, ? examples/s]

## 4. LoRa Configuration

In [5]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForSequenceClassification

base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Note that after hyperparameter search, using the suggested parameter does not boost the result much
# Thus the same values from the draft are kept
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    task_type=TaskType.SEQ_CLS,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_lin", "v_lin"]  # specific to DistilBERT
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


## 5. Trainer Arguments Configuration

In [ ]:
# same parameters are used, same rationale
training_args = TrainingArguments(
    output_dir="./results_distilbert_final",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=1
)

# Metric function
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }


## 6. Training

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


C:\Users\Dayso\AppData\Local\Temp\ipykernel_41080\2852150839.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.111700,0.105638,0.970027,0.970931
2,0.031800,0.080780,0.979799,0.980235
3,0.077500,0.095695,0.976933,0.977565


c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\other.py:1221: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8d3d091d-4ef8-47be-b2eb-cfc0bb5b6ae8)') - silently ignoring the lookup for the file config.json in distilbert-base-uncased.
  warnings.warn(
c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\save_and_load.py:238: UserWarning: Could not find a config file in distilbert-base-uncased - will assume that the vocabulary was not modified.
  warnings.warn(
c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\other.py:1221: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 29150e52-999e-4560-a3c1-92c8046051dc)') - silently ignoring the lookup for 

TrainOutput(global_step=15966, training_loss=0.0796519636671062, metrics={'train_runtime': 1315.4831, 'train_samples_per_second': 194.169, 'train_steps_per_second': 12.137, 'total_flos': 1.7207973866778624e+16, 'train_loss': 0.0796519636671062, 'epoch': 3.0})

## 7. Evaluation with test set

In [8]:
# Final eval
trainer.evaluate()

# Classification report
preds = trainer.predict(tokenized["test"])
y_pred = np.argmax(preds.predictions, axis=1)
y_true = preds.label_ids

print(classification_report(y_true, y_pred, target_names=["Human", "AI"]))


              precision    recall  f1-score   support

       Human       0.99      0.97      0.98     10508
          AI       0.97      0.99      0.98     10778

    accuracy                           0.98     21286
   macro avg       0.98      0.98      0.98     21286
weighted avg       0.98      0.98      0.98     21286



## 8. Saving the model

In [9]:
# === Save PEFT adapter + tokenizer ===
ADAPTER_DIR = "./saved_db/distilbert_peft_adapter_final"
TOKENIZER_DIR = ADAPTER_DIR  # save tokenizer alongside for convenience

# make sure trainer.model is your trained PEFT model
trainer.model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(TOKENIZER_DIR)

print(f"Saved PEFT adapter + tokenizer to: {ADAPTER_DIR}")

Saved PEFT adapter + tokenizer to: ./saved_db/distilbert_peft_adapter_final


In [10]:
# === Merge LoRA into base weights and save a standard model ===
from copy import deepcopy

MERGED_DIR = "./saved_db/distilbert_merged_full_final"

# work on a copy so the trainer's in-memory model keeps its PEFT structure
merged = deepcopy(trainer.model)
merged = merged.merge_and_unload()  # folds LoRA weights into the base model

merged.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)

print(f"Saved merged full model + tokenizer to: {MERGED_DIR}")

Saved merged full model + tokenizer to: ./saved_db/distilbert_merged_full_final
